**AUTHOR: RAIHAN SALMAN BAEHAQI (1103220180)**

**PART I**  

**The Fundamentals of Machine Learning**  

---

**CHAPTER 4 - Training Models**  

---

Chapter 4 delves into the inner workings of machine learning models, focusing on Linear Regression and its variants, Gradient Descent optimization, Polynomial Regression with regularization techniques, and classification models including Logistic and Softmax Regression.​  

---

**Linear Regression**  
Linear Regression makes predictions by computing a weighted sum of input features plus a bias term (intercept).  

Model Equations
Equation 4-1. Linear Regression model prediction  
![Eq4-1.jpg](./04.Chapter-04/Eq4-1.jpg)  

Where:
* $\hat{y}$ is the predicted value
* n is the number of features
* x<sub>i</sub> is the ith feature value
* θ<sub>j</sub> is the jth model parameter (including bias term θ<sub>0</sub> and feature weights θ<sub>1</sub>, θ<sub>2</sub>, ..., θ<sub>n</sub>)

Equation 4-2. Linear Regression model prediction (vectorized form)  
![Eq4-2.jpg](./04.Chapter-04/Eq4-2.jpg)  

Where:
* θ is the model's parameter vector containing bias and feature weights​
* x is the instance's feature vector with x<sub>0</sub> always equal to 1​
* θ⋅x is the dot product​
* h<sub>θ</sub> is the hypothesis function using model parameters θ

**Cost Function**  
Training means minimizing the Mean Squared Error (MSE) rather than RMSE for mathematical convenience.​

Equation 4-3. MSE cost function for a Linear Regression model  
![Eq4-3.jpg](./04.Chapter-04/Eq4-3.jpg)



**The Normal Equation**  

The closed-form solution directly computes optimal parameters.​

Equation 4-4. Normal Equation  
![Eq4-4.jpg](./04.Chapter-04/Eq4-4.jpg)  

Where:
* $\hat{θ}$ is the value of θ that minimizes the cost function​
* y is the vector of target values​

Generate test data:

In [ ]:
import numpy as np
X = 2 * np.random.rand(100, 1)
y = 4 + 3 * X + np.random.randn(100, 1)

![Figure4-1.jpg](./04.Chapter-04/Figure4-1.jpg)  

Compute θ using the Normal Equation:

In [ ]:
X_b = np.c_[np.ones((100, 1)), X]  # add x0 = 1 to each instance
theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)

>>> theta_best
array([[4.21509616],
       [2.77011339]])

The original function was y = 4 + 3x<sub>1</sub> + Gaussian noise and the algorithm found
θ<sub>0</sub> = 4.215 θ<sub>1</sub> = 2.770  

Make predictions:

In [ ]:
>>> X_new = np.array([[0], [2]])
>>> X_new_b = np.c_[np.ones((2, 1)), X_new]
>>> y_predict = X_new_b.dot(theta_best)
>>> y_predict
array([[4.21509616],
       [9.75532293]])

Plot predictions:

In [ ]:
plt.plot(X_new, y_predict, "r-")
plt.plot(X, y, "b.")
plt.axis([0, 2, 0, 15])
plt.show()

![Figure4-2.jpg](./04.Chapter-04/Figure4-2.jpg)  

**Using Scikit-Learn **
Perform Linear Regression simply:

In [ ]:
>>> from sklearn.linear_model import LinearRegression
>>> lin_reg = LinearRegression()
>>> lin_reg.fit(X, y)
>>> lin_reg.intercept_, lin_reg.coef_
(array([4.21509616]), array([[2.77011339]]))
>>> lin_reg.predict(X_new)
array([[4.21509616],
       [9.75532293]])

Scikit-Learn separates the bias term (intercept_) from feature weights (coef_).  

The LinearRegression class uses scipy.linalg.lstsq() (least squares):

In [ ]:
>>> theta_best_svd, residuals, rank, s = np.linalg.lstsq(X_b, y, rcond=1e-6)
>>> theta_best_svd
array([[4.21509616],
       [2.77011339]])

Or compute the pseudoinverse directly:

In [ ]:
>>> np.linalg.pinv(X_b).dot(y)
array([[4.21509616],
       [2.77011339]])

The pseudoinverse is computed using Singular Value Decomposition (SVD) that decomposes X into UΣV<sup>T</sup>. The pseudoinverse is X<sup>+</sup> = V Σ<sup>+</sup> U<sup>T</sup>. This approach handles edge cases where X<sup>T</sup> X is not invertible (singular).

**Computational Complexity**  
The Normal Equation computes the inverse of an (n + 1) × (n+ 1) matrix with complexity O(n<sup>2.4</sup>) to O(n<sup>3</sup>). Doubling features multiplies computation time by roughly 5.3 to 8.​

SVD approach is about O(n<sup>2</sup>). Both approaches get slow when features exceed 100,000 but are linear with regard to training instances O(m). Predictions are very fast: linear with both instances and features.  

---

**Gradient Descent**  
Gradient Descent is a generic optimization algorithm that iteratively tweaks parameters to minimize a cost function.​

The algorithm measures the local gradient of the error function with regard to parameter vector θ and moves in the direction of descending gradient. You start by filling θ with random values (random initialization), then improve it gradually until convergence.  

![Figure4-3.jpg](./04.Chapter-04/Figure4-3.jpg)

An important parameter is the learning rate hyperparameter that determines step size. If too small, the algorithm requires many iterations  

![Figure4-4.jpg](./04.Chapter-04/Figure4-4.jpg)  

If too high, you might jump across the valley and diverge  

![Figure4-5.jpg](./04.Chapter-04/Figure4-5.jpg)  

![Figure4-6.jpg](./04.Chapter-04/Figure4-6.jpg)  

Fortunately, the MSE cost function for Linear Regression is convex (any line segment joining two points never crosses the curve). This means there are no local minima, just one global minimum, and it's a continuous function with slope that never changes abruptly. Gradient Descent is guaranteed to approach the global minimum arbitrarily close.  

![Figure4-7.jpg](./04.Chapter-04/Figure4-7.jpg)  

Important: When using Gradient Descent, ensure all features have similar scale (e.g., using StandardScaler), or convergence will take much longer.


**Batch Gradient Descent**  
Compute partial derivatives of the cost function with regard to each parameter θ<sub>j</sub>.

Equation 4-5. Partial derivatives of the cost function  
![Eq4-5.jpg](./04.Chapter-04/Eq4-5.jpg)  

Equation 4-6. Gradient vector of the cost function  
![Eq4-6.jpg](./04.Chapter-04/Eq4-6.jpg)  

This formula involves calculations over the full training set X at each step, which is why it's called Batch Gradient Descent. It's terribly slow on very large training sets but scales well with number of features.​  

Equation 4-7. Gradient Descent step  
![Eq4-7.jpg](./04.Chapter-04/Eq4-7.jpg)  

Where η (eta) is the learning rate.​

Quick implementation:

In [ ]:
eta = 0.1  # learning rate
n_iterations = 1000
m = 100
theta = np.random.randn(2,1)  # random initialization

for iteration in range(n_iterations):
    gradients = 2/m * X_b.T.dot(X_b.dot(theta) - y)
    theta = theta - eta * gradients

>>> theta
array([[4.21509616],
       [2.77011339]])

The result matches the Normal Equation exactly.  

![Figure4-8.jpg](./04.Chapter-04/Figure4-8.jpg)  

To set the number of iterations, use a very large number but interrupt when the gradient vector becomes tiny (norm smaller than tolerance ϵ).  

**Convergence Rate**  
With a fixed learning rate, Batch Gradient Descent will eventually converge to the optimal solution, but it can take O(1/ϵ) iterations. If you divide tolerance by 10 for more precision, the algorithm may run about 10 times longer.

**Stochastic Gradient Descent**  
The main problem with Batch GD is using the whole training set at every step, making it very slow when the training set is large. Stochastic Gradient Descent picks a random instance at every step and computes gradients based only on that single instance.​

Working on a single instance makes the algorithm much faster and enables training on huge datasets as only one instance needs to be in memory. It can be implemented as an out-of-core algorithm.​

Due to its random nature, the cost function bounces up and down, decreasing only on average. It will end up very close to the minimum but continue bouncing around, never settling down  
![Figure4-9.jpg](./04.Chapter-04/Figure4-9.jpg)  

This randomness helps the algorithm jump out of local minima, giving it a better chance of finding the global minimum than Batch GD. One solution is to gradually reduce the learning rate using a learning schedule. Steps start out large (quick progress, escape local minima), then get smaller, allowing the algorithm to settle at the global minimum.​

Implementation:

In [ ]:
n_epochs = 50
t0, t1 = 5, 50  # learning schedule hyperparameters

def learning_schedule(t):
    return t0 / (t + t1)

theta = np.random.randn(2,1)  # random initialization

for epoch in range(n_epochs):
    for i in range(m):
        random_index = np.random.randint(m)
        xi = X_b[random_index:random_index+1]
        yi = y[random_index:random_index+1]
        gradients = 2 * xi.T.dot(xi.dot(theta) - yi)
        eta = learning_schedule(epoch * m + i)
        theta = theta - eta * gradients

>>> theta
array([[4.21076011],
       [2.74856079]])

By convention, we iterate by rounds of m iterations; each round is called an epoch.  

![Figure4-10.jpg](./04.Chapter-04/Figure4-10.jpg)  

**Important**: Training instances must be independent and identically distributed (IID). Shuffle instances during training to ensure parameters get pulled toward the global optimum on average. If not shuffled (e.g., sorted by label), SGD will optimize for one label at a time and not settle close to the global minimum.​

Using Scikit-Learn's SGDRegressor:

In [ ]:
from sklearn.linear_model import SGDRegressor
sgd_reg = SGDRegressor(max_iter=1000, tol=1e-3, penalty=None, eta0=0.1)
sgd_reg.fit(X, y.ravel())

>>> sgd_reg.intercept_, sgd_reg.coef_
(array([4.24365286]), array([2.8250878]))

**Mini-batch Gradient Descent**  
**Mini-batch Gradient Descent** computes gradients on small random sets of instances called mini-batches. The main advantage over Stochastic GD is performance boost from hardware optimization of matrix operations, especially with GPUs.​

The algorithm's progress is less erratic than Stochastic GD and walks closer to the minimum, but it may be harder to escape local minima.  

![Figure4-11.jpg](./04.Chapter-04/Figure4-11.jpg)  

Algorithm Comparison  

Table 4-1. Comparison of algorithms for Linear Regression  
![Table4-1.jpg](./04.Chapter-04/Table4-1.jpg)  

Where m is number of training instances and n is number of features. After training, all algorithms end up with very similar models and make predictions identically.  

---

**Polynomial Regression**  

For data more complex than a straight line, add powers of each feature as new features, then train a linear model on this extended set.​

Generate nonlinear data based on a quadratic equation:

In [ ]:
m = 100
X = 6 * np.random.rand(m, 1) - 3
y = 0.5 * X**2 + X + 2 + np.random.randn(m, 1)

![Figure4-12.jpg](./04.Chapter-04/Figure4-12.jpg)  

Transform training data:

In [ ]:
>>> from sklearn.preprocessing import PolynomialFeatures
>>> poly_features = PolynomialFeatures(degree=2, include_bias=False)
>>> X_poly = poly_features.fit_transform(X)
>>> X[0]
array([-0.75275929])
>>> X_poly[0]
array([-0.75275929, 0.56664654])

X_poly now contains the original feature plus its square.​

Fit a LinearRegression model:

In [ ]:
>>> lin_reg = LinearRegression()
>>> lin_reg.fit(X_poly, y)
>>> lin_reg.intercept_, lin_reg.coef_
(array([1.78134581]), array([[0.93366893, 0.56456263]]))

![Figure4-13.jpg](./04.Chapter-04/Figure4-13.jpg)  

With multiple features, PolynomialFeatures adds all combinations up to the given degree. For two features a and b, degree=3 would add a<sup>2</sup>, a<sup>3</sup>, b<sup>2</sup>, b<sup>3</sup>, ab, a<sup>2</sup>b, ab<sup>2</sup>.  

Warning: PolynomialFeatures(degree=d) transforms an array containing n features into (n+d)!/d!n! features. Beware of combinatorial explosion.  

---

**Learning Curves**  
High-degree Polynomial Regression fits training data much better than plain Linear Regression.  

![Figure4-14.jpg](./04.Chapter-04/Figure4-14.jpg)  

Learning curves are plots of model performance on training and validation sets as a function of training set size.​

Function to plot learning curves:

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

def plot_learning_curves(model, X, y):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
    train_errors, val_errors = [], []
    for m in range(1, len(X_train)):
        model.fit(X_train[:m], y_train[:m])
        y_train_predict = model.predict(X_train[:m])
        y_val_predict = model.predict(X_val)
        train_errors.append(mean_squared_error(y_train[:m], y_train_predict))
        val_errors.append(mean_squared_error(y_val, y_val_predict))
    plt.plot(np.sqrt(train_errors), "r-+", linewidth=2, label="train")
    plt.plot(np.sqrt(val_errors), "b-", linewidth=3, label="val")

Plot for Linear Regression:

In [ ]:
lin_reg = LinearRegression()
plot_learning_curves(lin_reg, X, y)

![Figure4-15.jpg](./04.Chapter-04/Figure4-15.jpg)  

**Key insight**: If your model is underfitting, adding more training examples will not help. You need a more complex model or better features.  

10th-degree polynomial model:

In [ ]:
from sklearn.pipeline import Pipeline
polynomial_regression = Pipeline([
        ("poly_features", PolynomialFeatures(degree=10, include_bias=False)),
        ("lin_reg", LinearRegression()),
    ])
plot_learning_curves(polynomial_regression, X, y)

![Figure4-16.jpg](./04.Chapter-04/Figure4-16.jpg)  

**Key insight**: One way to improve an overfitting model is to feed it more training data until the validation error reaches the training error.  

**The Bias/Variance Trade-off**  
A model's generalization error can be expressed as the sum of three errors:​

**Bias**: Due to wrong assumptions (e.g., assuming data is linear when it's quadratic). High-bias models likely underfit training data.​

**Variance**: Due to excessive sensitivity to small variations in training data. Models with many degrees of freedom (high-degree polynomials) have high variance and overfit.​

**Irreducible error**: Due to data noisiness. Reduce this by cleaning data (fix sensors, remove outliers).​

Increasing model complexity typically increases variance and reduces bias. Conversely, reducing complexity increases bias and reduces variance.  

---

**Regularized Linear Models**  
A good way to reduce overfitting is to regularize the model (constrain it). Fewer degrees of freedom make it harder to overfit. For polynomial models, reduce polynomial degrees. For linear models, constrain the weights.

**Ridge Regression**  
**Ridge Regression** (Tikhonov regularization) adds a regularization term $
\alpha \sum_{i=1}^{n} \theta_i^2
$ to the cost function. This forces the learning algorithm to fit the data and keep model weights small.​

**Important**: The regularization term should only be added during training. Use the unregularized performance measure for testing.​

The hyperparameter α controls regularization strength. If α = 0, Ridge Regression is just Linear Regression. If α is very large, all weights end up near zero (flat line through data's mean).  

Equation 4-8. Ridge Regression cost function  
![Eq4-8.jpg](./04.Chapter-04/Eq4-8.jpg)  

The bias term θ<sub>0</sub> is not regularized. The regularization term equalsals 1/2 (∥w∥<sub>2</sub>)<sup>2</sup>, where ∥w∥<sub>2</sub> is the ℓ2 norm of the weight vector  

Important: Scale the data (e.g., using StandardScaler) before performing Ridge Regression, as it's sensitive to input feature scale. This is true for most regularized models.  

![Figure4-17.jpg](./04.Chapter-04/Figure4-17.jpg)  

Equation 4-9. Ridge Regression closed-form solution  
![Eq4-9.jpg](./04.Chapter-04/Eq4-9.jpg)  

Where A is the (n+1)×(n+1) identity matrix with a 0 in the top-left cell (corresponding to bias term).​

Using Scikit-Learn with closed-form solution:

In [ ]:
>>> from sklearn.linear_model import Ridge
>>> ridge_reg = Ridge(alpha=1, solver="cholesky")
>>> ridge_reg.fit(X, y)
>>> ridge_reg.predict([[1.5]])
array([[1.55071465]])

Using Stochastic Gradient Descent:

In [ ]:
>>> sgd_reg = SGDRegressor(penalty="l2")
>>> sgd_reg.fit(X, y.ravel())
>>> sgd_reg.predict([[1.5]])
array([1.47012588])

The penalty hyperparameter sets the regularization term type. Specifying "l2" adds half the square of the ℓ2 norm of the weight vector (Ridge Regression).

**Lasso Regression**  
**Lasso Regression** (Least Absolute Shrinkage and Selection Operator) adds a regularization term using the ℓ1 norm of the weight vector.  

Equation 4-10. Lasso Regression cost function  
![Eq4-10.jpg](./04.Chapter-04/Eq4-10.jpg)  

![Figure4-18.jpg](./04.Chapter-04/Figure4-18.jpg)  

An important characteristic: Lasso tends to eliminate weights of least important features (set them to zero). For example, the dashed line with α=10<sup>−7</sup> looks quadratic, almost linear—all high-degree polynomial feature weights equal zero. Lasso automatically performs feature selection and outputs a sparse model.  

![Figure4-19.jpg](./04.Chapter-04/Figure4-19.jpg)  

Important: To avoid bouncing around the optimum with Lasso, gradually reduce the learning rate during training.​

The Lasso cost function is not differentiable at θ<sub>i</sub> = 0, but Gradient Descent works fine using a subgradient vector.  

Equation 4-11. Lasso Regression subgradient vector  
![Eq4-11.jpg](./04.Chapter-04/Eq4-11.jpg)  

Using Scikit-Learn:

In [ ]:
>>> from sklearn.linear_model import Lasso
>>> lasso_reg = Lasso(alpha=0.1)
>>> lasso_reg.fit(X, y)
>>> lasso_reg.predict([[1.5]])
array([1.53788174])

**Elastic Net**  
**Elastic Net** is a middle ground between Ridge and Lasso. The regularization term is a mix of both, controlled by mix ratio r. When r=0, Elastic Net is equivalent to Ridge; when r=1, it's equivalent to Lasso.  

Equation 4-12. Elastic Net cost function  
![Eq4-12.jpg](./04.Chapter-04/Eq4-12.jpg)  

When to use which:
* It's almost always preferable to have at least a little regularization, so avoid plain Linear Regression​
* Ridge is a good default​
* Lasso or Elastic Net if you suspect only a few features are useful (they reduce useless feature weights to zero)​
* Elastic Net is preferred over Lasso because Lasso may behave erratically when features outnumber training instances or when several features are strongly correlated​

Using Scikit-Learn:

In [ ]:
>>> from sklearn.linear_model import ElasticNet
>>> elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.5)
>>> elastic_net.fit(X, y)
>>> elastic_net.predict([[1.5]])
array([1.54333232])

**Early Stopping**  
A different way to regularize iterative learning algorithms: stop training as soon as validation error reaches a minimum.  

![Figure4-20.jpg](./04.Chapter-04/Figure4-20.jpg)  

**Important**: With Stochastic and Mini-batch GD, curves aren't smooth. Stop only after validation error has been above the minimum for some time, then roll back parameters to where validation error was minimum.​

Basic implementation:

In [ ]:
from sklearn.base import clone

# prepare the data
poly_scaler = Pipeline([
        ("poly_features", PolynomialFeatures(degree=90, include_bias=False)),
        ("std_scaler", StandardScaler())
    ])
X_train_poly_scaled = poly_scaler.fit_transform(X_train)
X_val_poly_scaled = poly_scaler.transform(X_val)

sgd_reg = SGDRegressor(max_iter=1, tol=-np.infty, warm_start=True,
                       penalty=None, learning_rate="constant", eta0=0.0005)

minimum_val_error = float("inf")
best_epoch = None
best_model = None
for epoch in range(1000):
    sgd_reg.fit(X_train_poly_scaled, y_train)  # continues where it left off
    y_val_predict = sgd_reg.predict(X_val_poly_scaled)
    val_error = mean_squared_error(y_val, y_val_predict)
    if val_error < minimum_val_error:
        minimum_val_error = val_error
        best_epoch = epoch
        best_model = clone(sgd_reg)

With warm_start=True, fit() continues training where it left off instead of restarting.  

---

**Logistic Regression**  
Logistic Regression (Logit Regression) is commonly used to estimate the probability that an instance belongs to a particular class. If estimated probability > 50%, the model predicts the instance belongs to that class (positive class, labeled "1"); otherwise it predicts it doesn't (negative class, labeled "0"). This makes it a binary classifier.​

**Estimating Probabilities**  
Just like Linear Regression, Logistic Regression computes a weighted sum of input features (plus bias), but instead of outputting the result directly, it outputs the logistic of this result.  

Equation 4-13. Logistic Regression model estimated probability (vectorized form)  
![Eq4-13.jpg](./04.Chapter-04/Eq4-13.jpg)  

The logistic—noted σ(⋅)—is a sigmoid function (S-shaped) that outputs a number between 0 and 1.  

Equation 4-14. Logistic function  
![Eq4-14.jpg](./04.Chapter-04/Eq4-14.jpg)  

![Figure4-21.jpg](./04.Chapter-04/Figure4-21.jpg)  

Once the model has estimated probability $\hat{p}$ = h<sub>θ</sub>(x), it can make predictions easily.  

Equation 4-15. Logistic Regression model prediction  
![Eq4-15.jpg](./04.Chapter-04/Eq4-15.jpg)  

Notice that σ(t)<0.5 when t<0, and σ(t)≥0.5 when t≥0, so Logistic Regression predicts 1 if x<sup>T</sup>θ is positive and 0 if negative.​

The score t is often called the logit. The name comes from the logit function logit (p)= log(p/(1−p)), which is the inverse of the logistic function. The logit is also called the log-odds (log of the ratio between estimated probability for positive class and estimated probability for negative class).


**Training and Cost Function**  

The training objective is to set parameter vector θ so the model estimates high probabilities for positive instances (y=1) and low probabilities for negative instances (y=0).  

Equation 4-16. Cost function of a single training instance  
![Eq4-16.jpg](./04.Chapter-04/Eq4-16.jpg)  

This makes sense because −log(t) grows very large when t approaches 0. The cost will be large if the model estimates a probability close to 0 for a positive instance or close to 1 for a negative instance.  

Equation 4-17. Logistic Regression cost function (log loss)  
![Eq4-17.jpg](./04.Chapter-04/Eq4-17.jpg)  

There is no closed-form equation to compute θ that minimizes this cost function. However, the cost function is convex, so Gradient Descent is guaranteed to find the global minimum.  

Equation 4-18. Logistic cost function partial derivatives  
![Eq4-18.jpg](./04.Chapter-04/Eq4-18.jpg)  

This equation looks very much like the Linear Regression gradient: for each instance it computes the prediction error and multiplies it by the jth feature value, then computes the average over all training instances.

**Decision Boundaries**  
Use the iris dataset to illustrate Logistic Regression. The dataset contains sepal and petal length and width of 150 iris flowers of three different species: Iris setosa, Iris versicolor, and Iris virginica.  

![Figure4-22.jpg](./04.Chapter-04/Figure4-22.jpg)  

Build a classifier to detect Iris virginica based only on petal width:

In [ ]:
>>> from sklearn import datasets
>>> iris = datasets.load_iris()
>>> list(iris.keys())
['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename']
>>> X = iris["data"][:, 3:]  # petal width
>>> y = (iris["target"] == 2).astype(np.int)  # 1 if Iris virginica, else 0

Train a Logistic Regression model:

In [ ]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(X, y)

Look at estimated probabilities for petal widths from 0 cm to 3 cm:

In [ ]:
X_new = np.linspace(0, 3, 1000).reshape(-1, 1)
y_proba = log_reg.predict_proba(X_new)
plt.plot(X_new, y_proba[:, 1], "g-", label="Iris virginica")
plt.plot(X_new, y_proba[:, 0], "b--", label="Not Iris virginica")

![Figure4-23.jpg](./04.Chapter-04/Figure4-23.jpg)  

Make predictions:

In [ ]:
>>> log_reg.predict([[1.7], [1.5]])
array([1, 0])

Display two features: petal width and length.  

![Figure4-24.jpg](./04.Chapter-04/Figure4-24.jpg)  

Logistic Regression models can be regularized using ℓ1 or ℓ2 penalties. Scikit-Learn adds an ℓ2 penalty by default.​

**Important**: The hyperparameter controlling regularization strength is not alpha but its inverse: C. The higher the value of C, the less the model is regularized.

Softmax Regression
Logistic Regression can be generalized to support multiple classes directly, without training and combining multiple binary classifiers. This is called Softmax Regression or Multinomial Logistic Regression.​

When given an instance x, the model first computes a score s<sub>k</sub>(x) for each class k, then estimates the probability of each class by applying the softmax function to the scores.  

Equation 4-19. Softmax score for class k  
![Eq4-19.jpg](./04.Chapter-04/Eq4-19.jpg)  

Each class has its own dedicated parameter vector θ<sup>(k)</sup>. All vectors are typically stored as rows in a parameter matrix Θ.  

Equation 4-20. Softmax function  
![Eq4-20.jpg](./04.Chapter-04/Eq4-20.jpg)  

Where:
* K is the number of classes​
* s(x) is a vector containing scores of each class for instance x
* σ(s(x))<sub>k</sub> is the estimated probability that instance x belongs to class k​

Scores are generally called logits or log-odds (unnormalized).  

![Eq4-21.jpg](./04.Chapter-04/Eq4-21.jpg)  

The argmax operator returns the value of a variable that maximizes a function. The classifier predicts only one class at a time (multiclass, not multioutput).​

**Important**: Use only with mutually exclusive classes (different types of plants). Cannot use to recognize multiple people in one picture.​

The training objective is to have a model that estimates a high probability for the target class and low probability for other classes.  

Equation 4-22. Cross entropy cost function  
![Eq4-22.jpg](./04.Chapter-04/Eq4-22.jpg)  

Where:
* y<sub>k</sub><sup>(i)</sup> is the target probability that the ith instance belongs to class k. Generally equal to 1 or 0 depending on whether the instance belongs to the class​

When there are just two classes (K=2), this is equivalent to Logistic Regression's log loss.  

**Cross Entropy**  
Cross entropy originated from information theory. If you want to efficiently transmit weather information with eight options, you could encode each using three bits (2<sup>3</sup> = 8). However, if "sunny" is most common, it's more efficient to code "sunny" with one bit (0) and other seven options with four bits (starting with 1). Cross entropy measures the average number of bits you actually send per option.​

If your assumption is perfect, cross entropy equals the entropy of the weather itself (intrinsic unpredictability). If assumptions are wrong, cross entropy will be greater by an amount called the Kullback-Leibler (KL) divergence.​

Cross entropy between two probability distributions p and q is defined as H(p,q)=−∑<sub>x</sub> p(x) log q(x).  

Equation 4-23. Cross entropy gradient vector for class k  
![Eq4-23jpg](./04.Chapter-04/Eq4-23.jpg)  

Compute the gradient vector for every class, then use Gradient Descent to find parameter matrix Θ that minimizes the cost function.​

**Using Scikit-Learn**  
Classify iris flowers into all three classes. LogisticRegression uses one-versus-the-rest by default for more than two classes, but you can set multi_class to "multinomial" to switch to Softmax Regression. Must specify a solver supporting Softmax Regression like "lbfgs". It applies ℓ2 regularization by default, controlled by hyperparameter C:

In [ ]:
X = iris["data"][:, (2, 3)]  # petal length, petal width
y = iris["target"]

softmax_reg = LogisticRegression(multi_class="multinomial",solver="lbfgs", C=10)
softmax_reg.fit(X, y)

Make predictions:

In [ ]:
>>> softmax_reg.predict([[5, 2]])
array([2])
>>> softmax_reg.predict_proba([[5, 2]])
array([[6.38014896e-07, 5.74929995e-02, 9.42506362e-01]])

For an iris with 5 cm long and 2 cm wide petals, the model predicts Iris virginica (class 2) with 94.2% probability (or Iris versicolor with 5.8% probability).  

![Figure4-25.jpg](./04.Chapter-04/Figure4-25.jpg)
